In [8]:
# download dataset
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [9]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as file:
    text = file.read()

print(f"length of dataset in characters {len(text)}")

length of dataset in characters 1115394


In [10]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [11]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(chars)
print(''.join(chars))
print(vocab_size)

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [12]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars)}
itos = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string
 
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [13]:
print(list(map(encode, "hii there")))

[[46], [47], [47], [1], [58], [46], [43], [56], [43]]


In [14]:
numbers = [1, 2, 3, 4, 5]

# Using a lambda function with map to square each number
squared_numbers_map_object = map(lambda x: x * x, numbers)

print(list(squared_numbers_map_object))

[1, 4, 9, 16, 25]


In [15]:
if False:
    import tiktoken
    enc = tiktoken.get_encoding("gpt2")
    print(enc.n_vocab)
    enc.encode("hii there")
# GPT trades of the code book size and the length of sequence
# tiktoken book ranges from 0 to 50256, hii there len is 3 instead 9

In [16]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # use PyTorch
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[0:1000]) # the 1000 characters we looked at earlier will to the GPT look like thi

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [17]:
# let's now split up the data into train and validation sets
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [18]:
block_size = 8
train_data[:block_size+1]
# in the case of 18, 47 comes next
# in the case of 18, 47, ... 56 comes next
# in the case of 18, 46, 56, ... 57 comes next
# ...
# 8 training cases

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [28]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the targe is {target}")
# context size varies, so that transformer is used to see block of different sizes
# transformer is not able to handle sequence longer than block size. As it has never seen it

when input is tensor([18]) the targe is 47
when input is tensor([18, 47]) the targe is 56
when input is tensor([18, 47, 56]) the targe is 57
when input is tensor([18, 47, 56, 57]) the targe is 58
when input is tensor([18, 47, 56, 57, 58]) the targe is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the targe is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the targe is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the targe is 58


In [108]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallerl?
block_size = 8 # what is maximum context length for predictions

def get_batch(split):
    # generate a small batch of input x and targets y
    data = train_data if split == "train" else val_data
    ix  = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('target:')
print(yb.shape)
print(yb)
print('--------')

for b in range(batch_size): # batch_dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
target:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
--------
when input is tensor([24]) the target is 43
when input is tensor([24, 43]) the target is 58
when input is tensor([24, 43, 58]) the target is 5
when input is tensor([24, 43, 58,  5]) the target is 57
when input is tensor([24, 43, 58,  5, 57]) the target is 1
when input is tensor([24, 43, 58,  5, 57,  1]) the target is 46
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target is 43
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target is 39
when input is tensor([44]) the target is 53
when input is tensor([44, 53]) the target is 56
when input is tensor([44, 53, 56]) the target

In [101]:
torch.randint(1000, (2,5))

tensor([[295, 753,  15,  10, 834],
        [ 90, 212, 120, 297, 286]])

In [102]:
print(xb) # our input to transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [103]:
# below is an example of embedding: which converts 0-9 integer to (1,3) vector
# embedding of size (10, 3) create 10 tensors of size 3
# embedding(2) then reads the 2nd tensor of the embedding size (1, 3)
embedding = nn.Embedding(10, 3)
print(embedding.weight)
input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 9]])
print(input)
embedding(input)

Parameter containing:
tensor([[ 1.6305,  0.7277,  1.3488],
        [-0.1396,  0.2858,  0.9651],
        [-2.0371,  0.4931, -1.2435],
        [-0.1680,  0.1260, -1.5627],
        [-1.1601, -0.3348,  0.1526],
        [ 0.3843,  1.3091,  0.4645],
        [ 1.6455, -0.8030,  1.3514],
        [-0.2759, -1.7097, -0.0826],
        [-0.0495, -1.4480,  1.4516],
        [-1.5103,  0.8212, -0.2115]], requires_grad=True)
tensor([[1, 2, 4, 5],
        [4, 3, 2, 9]])


tensor([[[-0.1396,  0.2858,  0.9651],
         [-2.0371,  0.4931, -1.2435],
         [-1.1601, -0.3348,  0.1526],
         [ 0.3843,  1.3091,  0.4645]],

        [[-1.1601, -0.3348,  0.1526],
         [-0.1680,  0.1260, -1.5627],
         [-2.0371,  0.4931, -1.2435],
         [-1.5103,  0.8212, -0.2115]]], grad_fn=<EmbeddingBackward0>)

In [104]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        #each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        # The token embedding table is vocab_size by vocab_size
       #  Parameter containing:
       #  tensor([[-0.9873, -0.1692, -1.7186,  ...,  0.4657, -1.2951,  0.0595],
       #  [-1.1337,  0.4521, -2.1238,  ..., -1.6573, -0.8691, -0.9001],
       #  [ 0.2113,  1.1337, -1.1722,  ..., -0.6102,  2.3405,  0.6097],
       #  ...,
       #  [ 0.3448, -0.6760, -0.2725,  ..., -0.7113, -0.6573, -1.6034],
       #  [ 0.4921, -0.2105,  0.2333,  ..., -0.2867,  0.2278, -1.6450],
       #  [-1.1449, -1.7441,  0.5990,  ...,  0.8644,  0.2535, -0.9051]],
       # requires_grad=True)
       # for example, char (1) after embedding gives [-1.1337,  0.4521, -2.1238,  ..., -1.6573, -0.8691, -0.9001]

    def forward(self, idx, targets = None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C) ---batch by time by channel
        # (B, T, C) = (8, 4, 65) we will black out these rows ranges them in 8,4,65
        # because idx(x) has shape of 4*8, after embedding, it has size of 4*8*65

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            # cross entropy requires logits to have a shape of B(4), C(65), T(8)
            logits = logits.view(B*T, C) # squence 4 by 8 by 65 into 32 by 65
            # print(logits)
            targets = targets.view(B*T)
            # print(targets)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is x size (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # (B, T, C) becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sampel from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled indexs to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B,  T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m.forward(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long)
print(decode(idx[0].tolist()))
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)



SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [105]:
# create a Pytorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3) # general learning rate is 1e-4, but for simple model it can be faster

In [ ]:
batch_size = 32
for steps in range(10000):

    # sample a batch of data
    xb, yb = get_batch('train')
    # print(xb)
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True) # zeroing out grad in previous step
    loss.backward()  #get new gradient for all parameters
    optimizer.step() #use the gradient to update the parameter

print(loss.item())

In [127]:
print(decode(m.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=500)[0].tolist()))




G Yerposhedamede: my the gherfan'sthea mstl folfe wer d by-
'd bl hol we t thowhavat INayenchequre
S:
Cowis ang alawhepevecadunde? my ay homean:
ho g terongod D hofe Hes umeso ger theallatcouse wert my, athy, mes ins, AROfin, mye asth.
MICA f mumateterd t e p for t ptwe bor hethther shyo he owimaldizes mond w, ougr! t caphincy ou trvesirorutle d lu galactinug foulimsteirve mite.
Whiow herelpoureory ithit; t to---gr whomy, I 'lefanowigir e mourge od sothodyote
M: ou, llkipathrss, d ngead ilif a


## The mathmetical trick in self-attention 

In [131]:
# consider the folloing toy example:

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape
print(x)
# the 8 tokens do not talk to each other, but we want them to talk to each other

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]])


In [135]:
# we want x[b, t] = mean_{i<=t} x[b,i[
xbow = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0) #0 dimension is the time dimension
print(xbow)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


In [143]:
torch.manual_seed(42)
a = torch.ones(3, 3)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim = True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b  # matrix multiplication:  3*3 by 3*2 = 3*2
print('a=')
print(a)
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [151]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow = torch.zeros(B, T, C)
for b in range(B):
    xbow[b] = wei @ x[b]
xbow2 = wei @ x # size: (T by T) @ (B by T by C) -> (B, T, T) @ (B, T, C) -> (B, T, C)
print(xbow2)
torch.allclose(xbow, xbow2)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


True

In [160]:
# version 3: use softmax
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros(T, T) # how much each token in the past should be used
wei = wei.masked_fill(tril == 0, float('-inf')) # token in future cannot talk to the past 
wei = F.softmax(wei, dim=1)
# print(wei)
xbow3 = wei @ x
print(xbow3)

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])


- every single node at each position will emit two vectors: query & key
  - query vector: is what I am looking for
  - what do I contain
- Do a dot product between keys and querys

In [185]:
# version 4: self-attention
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channel
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias = False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = query(x) # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1)  # (B, T, 16) @ (B, 16, T) = (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v # (B, T, T) @ (B, T, 16) --> (B, T, 16)

# out = wei @ x # (B, T, T) @ (B, T, C) --> (B, T, C)

out.shape

torch.Size([4, 8, 16])

## Notes:
* Attention is a communication mechasim. Can be seen as nodes in a directed graph looking at each other and aggregating
* There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens

# Layer norm

In [188]:
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        # calculate the forward pass
        xmean = x.mean(1, keepdim=True) # batch mean
        xvar = x.var(1, keepdim=True) # batch variance
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return (self.gamma, self.beta)

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape
print(x[0,:].mean())
print(x[0,:].var())
print(x[0,:])

tensor(2.3842e-09)
tensor(1.0000)
tensor([ 0.1335, -0.1059, -0.3824, -0.9127,  0.5583, -0.0147,  0.8721,  0.0223,
         0.3057,  1.0758, -1.3277, -0.5261,  0.1862, -0.2680, -0.9184,  1.4341,
         1.2485, -0.1724,  0.2337,  0.8822, -1.9837,  0.4317,  1.3804,  0.5251,
         0.0812, -1.5308, -1.1465, -0.3587,  0.3884, -0.8043,  1.4153,  2.3556,
        -0.6721, -0.2789,  0.9252,  0.0769,  0.1121,  1.0434, -1.1406, -0.3239,
        -0.5236, -0.9210,  0.4828, -1.4660, -1.1900,  0.5068, -0.6093, -0.7013,
         1.5317, -0.8056,  1.2509, -0.3025, -1.4813,  1.9701,  2.5984, -1.7063,
         1.3466, -1.4808,  0.7448, -0.2410,  0.7044,  1.4245,  1.4975, -0.4240,
        -0.8357,  0.5316, -0.0882, -0.1007, -0.5135,  0.4055, -0.2847, -1.0579,
         1.8786, -0.5517,  0.1736,  0.6263, -1.4009,  0.8257,  0.0989,  0.1786,
         2.3378, -1.2072,  0.9257,  0.4917, -1.5602, -1.2520,  0.6199, -0.2263,
         0.2902, -0.3654,  0.4152, -0.7706, -0.4390, -0.4562, -0.9948, -0.4299,
      

In [162]:
position_embedding_tb = nn.Embedding(8, 32)

In [163]:
print(position_embedding_tb)

Embedding(8, 32)


In [170]:
x = torch.randint(0,8, (4,8))
print(x)

tensor([[6, 4, 5, 2, 4, 6, 4, 4],
        [4, 1, 1, 2, 0, 4, 5, 0],
        [5, 0, 2, 4, 4, 3, 5, 0],
        [0, 7, 7, 1, 3, 6, 7, 6]])


In [171]:
position_embedding_tb(x)

tensor([[[-0.6421,  0.8266, -0.3646,  ..., -0.6845,  0.7134,  0.5298],
         [-0.6013, -0.0996, -1.2311,  ...,  0.7546,  0.4145,  1.3664],
         [-0.0462, -1.6457, -0.6960,  ...,  0.0916,  1.5164, -0.4389],
         ...,
         [-0.6421,  0.8266, -0.3646,  ..., -0.6845,  0.7134,  0.5298],
         [-0.6013, -0.0996, -1.2311,  ...,  0.7546,  0.4145,  1.3664],
         [-0.6013, -0.0996, -1.2311,  ...,  0.7546,  0.4145,  1.3664]],

        [[-0.6013, -0.0996, -1.2311,  ...,  0.7546,  0.4145,  1.3664],
         [-0.7150, -0.0476,  0.5230,  ..., -1.4120,  1.7361,  1.8350],
         [-0.7150, -0.0476,  0.5230,  ..., -1.4120,  1.7361,  1.8350],
         ...,
         [-0.6013, -0.0996, -1.2311,  ...,  0.7546,  0.4145,  1.3664],
         [-0.0462, -1.6457, -0.6960,  ...,  0.0916,  1.5164, -0.4389],
         [-0.1250,  0.7821,  0.5635,  ...,  0.7337,  0.2510,  0.0770]],

        [[-0.0462, -1.6457, -0.6960,  ...,  0.0916,  1.5164, -0.4389],
         [-0.1250,  0.7821,  0.5635,  ...,  0

In [173]:
position_embedding_tb(torch.IntTensor(1))

tensor([[-0.1250,  0.7821,  0.5635,  1.8582,  1.0441, -0.8638,  0.8351, -0.3157,
         -1.9776,  0.0179, -1.4129, -1.8791, -0.1798,  0.7904,  0.5239, -0.2694,
          1.7093,  0.0579,  0.8637, -0.5890, -1.0340, -0.2179,  0.7987,  0.9105,
          0.2691, -0.0366, -0.4808,  0.3163,  0.3866,  0.7337,  0.2510,  0.0770]],
       grad_fn=<EmbeddingBackward0>)

In [177]:
torch.arange(8)

tensor([0, 1, 2, 3, 4, 5, 6, 7])

In [178]:
aa = torch.rand(4,8,5)
bb = torch.rand(8,5)

In [179]:
print(aa)
print(bb)
print(aa+bb)

tensor([[[0.9227, 0.5303, 0.1988, 0.9099, 0.7135],
         [0.8311, 0.1619, 0.7910, 0.1585, 0.9947],
         [0.2882, 0.8013, 0.6001, 0.6325, 0.4233],
         [0.7054, 0.2916, 0.0287, 0.3079, 0.8918],
         [0.3684, 0.6572, 0.3151, 0.8751, 0.7992],
         [0.6765, 0.2444, 0.0914, 0.5188, 0.2067],
         [0.9111, 0.0195, 0.7234, 0.9985, 0.7504],
         [0.6705, 0.0189, 0.9809, 0.4145, 0.0328]],

        [[0.9936, 0.2965, 0.4646, 0.9576, 0.1534],
         [0.1463, 0.5813, 0.4331, 0.6152, 0.0806],
         [0.5150, 0.2776, 0.2542, 0.0422, 0.7651],
         [0.5963, 0.0773, 0.8968, 0.6508, 0.5928],
         [0.2064, 0.5754, 0.9818, 0.8429, 0.1106],
         [0.9564, 0.5388, 0.7405, 0.8883, 0.9263],
         [0.1102, 0.9378, 0.1604, 0.5375, 0.1506],
         [0.3904, 0.4773, 0.4402, 0.4210, 0.5394]],

        [[0.9932, 0.7905, 0.7797, 0.7001, 0.8871],
         [0.4769, 0.5398, 0.6029, 0.0639, 0.0972],
         [0.5613, 0.3044, 0.4908, 0.3853, 0.5778],
         [0.8253, 0.3342, 0

In [190]:
aaa = ['b' for _ in range(10)]
print(aaa)

['b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b', 'b']
